# Импорт библиотек

In [2]:
!pip install catboost -q

In [26]:
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import mlflow
from mlflow.tracking import MlflowClient

from sklearn.pipeline import Pipeline

from catboost import CatBoostClassifier

# Импорт данных

### Для бэйслайна возмем основной датасет

In [6]:
df = pd.read_csv('../data/application_train.csv')
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df.shape

(307511, 122)

# Разведочный анализ

In [8]:
df.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,...,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,...,0.008130,0.000595,0.000507,0.000335,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,102790.175348,0.272419,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.013831,4363.988632,141275.766519,...,0.089798,0.024387,0.022518,0.018299,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189145.500000,0.000000,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.010006,-19682.000000,-2760.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278202.000000,0.000000,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15750.000000,-1213.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367142.500000,0.000000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12413.000000,-289.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.000000,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,365243.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


### Уникальные значения и константные значения

In [10]:
(df.nunique() == 1).sum()

0

### Пропуски

In [11]:
df.isna().sum()

SK_ID_CURR                        0
TARGET                            0
NAME_CONTRACT_TYPE                0
CODE_GENDER                       0
FLAG_OWN_CAR                      0
                              ...  
AMT_REQ_CREDIT_BUREAU_DAY     41519
AMT_REQ_CREDIT_BUREAU_WEEK    41519
AMT_REQ_CREDIT_BUREAU_MON     41519
AMT_REQ_CREDIT_BUREAU_QRT     41519
AMT_REQ_CREDIT_BUREAU_YEAR    41519
Length: 122, dtype: int64

Пропусков не сильно много, при предобработке заменим их на медиану

# Предобработка данных

Строим пайплайн предобработки с помощью ColumnTransformer

In [12]:
X = df.drop(['TARGET'], axis=1)
y = df['TARGET']

cat_features = X.describe(include='object').columns
num_features = X.describe().columns

In [13]:
cat_tranformer = Pipeline([
    ('inputer', SimpleImputer(strategy='most_frequent')),
])

num_transformer = Pipeline([
    ('inputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler(with_mean=False))
])


transformer = ColumnTransformer([
    ('cat', cat_tranformer, cat_features),
    ('num', num_transformer, num_features)
])


transformer.fit(X)

X = pd.DataFrame(transformer.transform(X), columns=transformer.get_feature_names_out())
# X.head()

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [14]:
cat_features = [f'cat__{col}' for col in cat_features]
model = CatBoostClassifier(verbose=1000, cat_features=cat_features,task_type="GPU")

Learning rate set to 0.0246
0:	learn: 0.6658910	total: 60.6ms	remaining: 1m
999:	learn: 0.2401249	total: 1m 9s	remaining: 0us


# MLFLOW

In [15]:
%%bash
export MLFLOW_REGISTRY_URI=../mlflow/

In [24]:
mlflow.set_tracking_uri("http://localhost:5001")
mlflow.set_experiment('Credit')
with mlflow.start_run():
    model.fit(X_train, y_train)
    print(model.predict_proba(X_test))

    # Логирование модели и параметров
    mlflow.log_param(
        'f1', f1_score(y_test, model.predict(X_test)))

    mlflow.sklearn.log_model(
    model,
    artifact_path="catbost_credit",
    registered_model_name=f"catbost_credit")

Learning rate set to 0.024904
0:	learn: 0.6656862	total: 45.4ms	remaining: 45.4s
999:	learn: 0.2392829	total: 45.2s	remaining: 0us
[[0.9641758  0.0358242 ]
 [0.94347872 0.05652128]
 [0.96229508 0.03770492]
 ...
 [0.92326359 0.07673641]
 [0.92832033 0.07167967]
 [0.81421547 0.18578453]]


2025/02/22 19:40:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Catbost Credit'.
2025/02/22 19:40:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Catbost Credit, version 1
Created version '1' of model 'Catbost Credit'.


🏃 View run ambitious-cod-537 at: http://localhost:5001/#/experiments/1/runs/37459f7441a146a5819d4c9377e8f1e5
🧪 View experiment at: http://localhost:5001/#/experiments/1


In [25]:
mlflow.get_artifact_uri()

'file:///C:/Users/Руслан/Desktop/jupyter/auto-pipeline-airflow-mlflow-master/mlflow/1/887dfd0dea474d96b85bb67be60fe856/artifacts'

In [30]:
def get_version_model(config_name, client):
    """
    Получение последней версии модели из MLFlow
    """
    dict_push = {}
    for count, value in enumerate(
        client.search_model_versions(f"name='{config_name}'")):
        # client.list_registered_models()):
        # Все версии модели
        dict_push[count] = value
    return dict_push

In [32]:
client = MlflowClient()
last_version = get_version_model('Catbost Credit', client)
last_version

{0: <ModelVersion: aliases=[], creation_timestamp=1740235201142, current_stage='None', description='', last_updated_timestamp=1740235201142, name='Catbost Credit', run_id='37459f7441a146a5819d4c9377e8f1e5', run_link='', source='file:///C:/Users/Руслан/Desktop/jupyter/auto-pipeline-airflow-mlflow-master/mlflow/1/37459f7441a146a5819d4c9377e8f1e5/artifacts/catbost_credit', status='READY', status_message=None, tags={}, user_id='', version='1'>}